In [25]:
import pandas as pd
import json
import re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [26]:
run_name = "union"
run_name = "union_mm"

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))


        # load JSONL
records_test = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_test.tsv") as f:
    for line in f:
        records_test.append(json.loads(line))

In [27]:
df_all = pd.DataFrame(records)
df_test = pd.DataFrame(records_test)
df_test.head()

,left,right,match,match_confidence
0,COL make VAL CESSNA COL model VAL 310 COL name...,COL make VAL TEXTRN COL model VAL 680 COL name...,0,0.999975
1,COL make VAL DIAMOND COL model VAL DA20 COL na...,COL make VAL DA COL model VAL 40 COL series VA...,0,0.999891
2,COL make VAL CESSNA COL model VAL 210 COL name...,COL make VAL BE COL model VAL 1900 COL series ...,0,0.999968
3,COL make VAL WACO COL model VAL YMF COL cert V...,COL make VAL WACO COL model VAL YMF COL series...,1,0.999904
4,COL make VAL PAC COL model VAL 750XL COL name ...,COL make VAL NAVION COL model VAL NAVION COL s...,0,0.999934


In [28]:
gold_test = pd.read_csv("../data/aircraft_er/" + run_name + "/test.txt", sep="\t", header=None, names=["left", "right", "gold"])

gold_all = pd.read_csv("../data/aircraft_er/" + run_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])
print(gold_all.head())

                                                left  \
0  COL make VAL BEECH COL model VAL 50 COL name V...   
1  COL make VAL COMMANDER COL model VAL 114 COL n...   
2  COL make VAL FOUND COL model VAL CENTENNIAL 10...   
3  COL make VAL BOEING COL model VAL 737 COL name...   
4  COL make VAL EMBRAER COL model VAL EMB120 COL ...   

                                               right  gold  
0  COL make VAL BE COL model VAL 50 COL series VA...     1  
1  COL make VAL BE COL model VAL 23 COL series VA...     0  
2  COL make VAL FBA COL model VAL 2 COL series VA...     0  
3  COL make VAL AGUSTA COL model VAL A109 COL ser...     0  
4  COL make VAL EMB COL model VAL 545 COL series ...     0  


In [29]:
df_all["gold"] = gold_all["gold"]
df_test["gold"] = gold_test["gold"]

In [30]:


#Test Data

y_true_test = df_test["gold"]
y_pred_test = df_test["match"]

print("Accuracy:", accuracy_score(y_true_test, y_pred_test))
print("\nClassification report:\n", classification_report(y_true_test, y_pred_test))
print("\nConfusion matrix:\n", confusion_matrix(y_true_test, y_pred_test))



with open("../aircraft_er_predictions/" + run_name + "_metrics_test.txt", "w") as f:
    print("Accuracy:", accuracy_score(y_true_test, y_pred_test), file=f)
    print("\nClassification report:\n", file=f)
    print(classification_report(y_true_test, y_pred_test), file=f)
    print("\nConfusion matrix:\n", file=f)
    print(confusion_matrix(y_true_test, y_pred_test), file=f)

Accuracy: 0.9787045252883763

Classification report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       815
           1       0.96      0.96      0.96       312

    accuracy                           0.98      1127
   macro avg       0.97      0.97      0.97      1127
weighted avg       0.98      0.98      0.98      1127


Confusion matrix:
 [[804  11]
 [ 13 299]]


In [31]:
#All Data

y_true = df_all["gold"]
y_pred = df_all["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))



with open("../aircraft_er_predictions/" + run_name + "_metrics_all.txt", "w") as f:
    print("Accuracy:", accuracy_score(y_true, y_pred), file=f)
    print("\nClassification report:\n", file=f)
    print(classification_report(y_true, y_pred), file=f)
    print("\nConfusion matrix:\n", file=f)
    print(confusion_matrix(y_true, y_pred), file=f)

Accuracy: 0.9848570353649361

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99      7519
           1       0.96      0.99      0.97      3113

    accuracy                           0.98     10632
   macro avg       0.98      0.99      0.98     10632
weighted avg       0.99      0.98      0.98     10632


Confusion matrix:
 [[7385  134]
 [  27 3086]]


In [32]:
errors = df[df["gold"] != df["match"]]
print(errors[["left","right","gold","match","match_confidence"]].head(20))

                                                  left  \
14   COL make VAL AIR TRACTOR COL model VAL AT501 C...   
92   COL make VAL STEARMAN COL model VAL B75 COL na...   
114  COL make VAL LAKE COL model VAL LA4 COL name V...   
228  COL make VAL CONSOLIDATED VULTEE COL model VAL...   
251  COL make VAL BEECH COL model VAL 200 COL name ...   
384  COL make VAL AEROCAR COL model VAL I COL cert ...   
405  COL make VAL EMBRAER COL model VAL EMB135 COL ...   
486  COL make VAL PITTS COL model VAL S2 COL name V...   
509  COL make VAL AERONCA COL model VAL 65TAF COL n...   
560  COL make VAL MORANE SAULNIER COL model VAL MS8...   
643  COL make VAL MOONEY COL model VAL M20TN COL na...   
690  COL make VAL AERO COMMANDER COL model VAL 500 ...   
691  COL make VAL MOONEY COL model VAL M20L COL nam...   
736  COL make VAL PILATUS COL model VAL PC6 COL nam...   
738  COL make VAL GULFSTREAM COL model VAL GULFSTRE...   
827  COL make VAL CHAMPION COL model VAL 7FC COL na...   
845  COL make 

In [33]:
errors

,left,right,match,match_confidence,gold
14,COL make VAL AIR TRACTOR COL model VAL AT501 C...,COL make VAL AT COL model VAL 501 COL series V...,0,0.993614,1
92,COL make VAL STEARMAN COL model VAL B75 COL na...,COL make VAL B COL model VAL 75 COL series VAL...,1,0.838936,0
114,COL make VAL LAKE COL model VAL LA4 COL name V...,COL make VAL LA COL model VAL 4 COL series VAL...,1,0.999922,0
228,COL make VAL CONSOLIDATED VULTEE COL model VAL...,COL make VAL PB COL model VAL Y5 COL series VA...,1,0.999893,0
251,COL make VAL BEECH COL model VAL 200 COL name ...,COL make VAL BE COL model VAL 300 COL series V...,1,0.999803,0
...,...,...,...,...,...
10379,COL make VAL MOONEY COL model VAL M20R COL nam...,COL make VAL MOONEY COL model VAL 20 COL serie...,0,0.957095,1
10441,COL make VAL WACO COL model VAL RNF COL cert V...,COL make VAL WACO COL model VAL R COL series V...,0,0.999709,1
10450,COL make VAL BELLANCA COL model VAL 31 42 COL ...,COL make VAL BL COL model VAL 149 COL series V...,1,0.695505,0
10541,COL make VAL BEECH COL model VAL 35 COL name V...,COL make VAL BE COL model VAL 35 COL series VA...,1,0.574446,0


In [34]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [35]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "gold": row["gold"],
        "predicted": row["match"],
        "confidence": row["match_confidence"]
    })

aligned = pd.DataFrame(parsed)

In [36]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)